In [14]:
import pandas as pd
import numpy as np
from pandas import DataFrame, Series

path_in = 'data'
path_out = 'data/temp02'
today = pd.Timestamp.today()


In [39]:
#Load files
CAPABILITIES = pd.read_pickle('data/temp01/CAPABILITIES.pkl')
WO_HEADER = pd.read_pickle('data/temp01/WO_HEADER.pkl')
VIP_LRU =  pd.read_pickle('data/temp01/VIP_LRU.pkl')
LOST_BIZ =  pd.read_pickle('data/temp01/LOST_BIZ.pkl')
PARTS_MASTER = pd.read_pickle('data/temp01/PARTS_MASTER.pkl')

WO_HEADER['COMMENT'] = np.NaN

#flag PN that are not in Parts Master (those wont generate nothing)
PARTS_MASTER.drop(columns = ['DESCRIPTION','PNM','CLASS','LIST_PRICE','PRICEDATE','CODE','MFG'],inplace = True)
PARTS_MASTER.drop_duplicates(subset = 'PN', inplace = True, keep = 'last')
WO_HEADER = pd.merge(WO_HEADER, PARTS_MASTER,  how='left', left_on='PN', right_on = 'PN')


#merge WO and capa
WO_HEADER = pd.merge(WO_HEADER, CAPABILITIES,  how='left', left_on=['PN','GEO_CODE'], right_on = ['PN','GEO_CODE'] )

#find top active shop for a PN
LIST_LRUS = pd.DataFrame(index = WO_HEADER['PN'].unique(), columns=['TOP_GEO_CODE'])
VOLUME_PER_GEO_CODE = WO_HEADER[WO_HEADER['IS_ACTIVE'] == 1].pivot_table(
    aggfunc=len, values='WONO', columns=['PN'], index='GEO_CODE')

for column in VOLUME_PER_GEO_CODE: #MAYBE CHANGE THIS FOR A PIVOT TABLE FOR FASTER PROCESSING
    LIST_LRUS['TOP_GEO_CODE'][column]= VOLUME_PER_GEO_CODE[column].idxmax() #assign top shop
    VOLUME_PER_GEO_CODE[column][VOLUME_PER_GEO_CODE[column].idxmax()] = np.NaN #change top shop value to 0 to then test next top shop

#bring number of open capa
LIST_LRUS = pd.merge(LIST_LRUS,
                     CAPABILITIES.pivot_table(aggfunc=sum,index='PN', values='IS_ACTIVE'),
                     left_index = True, right_index = True, how = 'left')

LIST_LRUS.rename(columns = {'IS_ACTIVE':'NB_ACTIVE_CAPS'}, inplace = True)

#bring top shop active shop
WO_HEADER = pd.merge(WO_HEADER, LIST_LRUS, right_index = True , how = 'left',left_on = 'PN')

#-------determine the best geo_code
WO_HEADER['ADJ_GEO_CODE'] = np.NaN
#--if active then keep
WO_HEADER.loc[WO_HEADER.IS_ACTIVE == 1, 'ADJ_GEO_CODE'] = WO_HEADER['GEO_CODE'] 
#--if not active and only one cap is open then bring it
#build table showing PN with only 1 active cap, and show that cap back in WO_HEADER
LIST_LRUS_ONE_CAP = pd.DataFrame(
    index = WO_HEADER[(WO_HEADER['IS_ACTIVE'] != 1) & (WO_HEADER['NB_ACTIVE_CAPS'] == 1)]['PN'].unique())
LIST_LRUS_ONE_CAP = pd.merge(
    LIST_LRUS_ONE_CAP,CAPABILITIES[CAPABILITIES['IS_ACTIVE']==1],how='left',left_index = True, right_on = 'PN')
LIST_LRUS_ONE_CAP.rename(inplace = True, columns ={'GEO_CODE':'ONLY_ACTIVE_GEO_CODE'})
LIST_LRUS_ONE_CAP.drop(columns=['IS_ACTIVE'], inplace = True)
WO_HEADER = pd.merge(WO_HEADER,LIST_LRUS_ONE_CAP,left_on='PN',right_on='PN', how = 'left')
WO_HEADER.loc[(WO_HEADER.IS_ACTIVE != 1) & (WO_HEADER.NB_ACTIVE_CAPS == 1),
              'ADJ_GEO_CODE'] = WO_HEADER['ONLY_ACTIVE_GEO_CODE']
#--if not active but multiple caps open then change to top geo code
WO_HEADER.loc[(WO_HEADER.IS_ACTIVE != 1) & (WO_HEADER.NB_ACTIVE_CAPS != 1), 'ADJ_GEO_CODE'] = WO_HEADER['TOP_GEO_CODE']

#remove unwanted rows (dont really delete rows, just add a comment)

WO_HEADER['COMMENT'] = np.where(WO_HEADER['ADJ_GEO_CODE'] == 'MIA GSTE', 'TO REMOVE: GEO_CODE IS GSTE', WO_HEADER['COMMENT'])
WO_HEADER['COMMENT'] = np.where(WO_HEADER['ADJ_GEO_CODE'].isnull(), 'TO REMOVE: NO ACTIVE CAPABILITY', WO_HEADER['COMMENT'])


#add GEO_CODE_UPPER columns
UPPER_GEO_CODE = DataFrame(columns=['GEO_CODE','UPPER_GEO_CODE'],
                           data =[['MIA MRO','BIC'],['PHX','BIC'],['SDF','BIC'],['DOR','ARO'],['MED MRO','ARO']])
WO_HEADER = WO_HEADER.merge(UPPER_GEO_CODE, left_on ='ADJ_GEO_CODE', right_on = 'GEO_CODE',how='left')

#drop unused columns
WO_HEADER = WO_HEADER.drop(columns=['GEO_CODE_x','IS_ACTIVE','TOP_GEO_CODE',
                                    'NB_ACTIVE_CAPS','ONLY_ACTIVE_GEO_CODE','GEO_CODE_y'])

#show VIPs
WO_HEADER = WO_HEADER .merge(VIP_LRU,left_on='PN',right_on='PN',how='left')
WO_HEADER.rename(columns={'GEO_CODE_UPPER':'VIP_STATUS','Date_entered':'VIP_STATUS_DATE'},inplace=True)

#remove Lost Biz (remove all occurences after termination date - TODO add warning to users to review aging items)
WO_HEADER = WO_HEADER.merge(LOST_BIZ,on=['PN','COMPANY_CODE','UPPER_GEO_CODE'],how='left')
LOST_BIZ.drop_duplicates(subset=['PN','COMPANY_CODE','UPPER_GEO_CODE'],inplace = True)
WO_HEADER.drop(columns=['Date entered'],inplace= True)
WO_HEADER['COMMENT'] = np.where(~WO_HEADER['Termination date'].isnull(), 'TO REMOVE: LOST BUSINESS', WO_HEADER['COMMENT'])
WO_HEADER['COMMENT'] = np.where(WO_HEADER['STC'].isnull(), 'TO WARN: LRUs PN NOT IN MASTER', WO_HEADER['COMMENT'])

#save as pickle
WO_HEADER.to_pickle('data/temp02/WO_HEADER.pkl')